# ES|QL concepts
## FOSS4G Europe - Mostar

July 2025

## Resources

* [Blog post announcement](https://www.elastic.co/blog/esql-elasticsearch-piped-query-language)
* [Documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql)
* [Reference](https://www.elastic.co/docs/reference/query-languages/esql)
* Webinar: [ES|QL: Search. Aggregate. Transform. Visualize. All with one query](https://www.elastic.co/virtual-events/cd-esql-search-aggregate-transform-visualize-all-with-one-query)
* Technical blog posts in [Search Labs](https://www.elastic.co/search-labs/blog/category/esql)
* [Run this notebook in Google Colaboratory](https://colab.research.google.com/github/jsanz/foss4g_europe_lab/blob/main/02-esql.ipynb)

## Setup

In [1]:
# Install required dependencies using uv, if available, or directly with pip
!pip install -qU elasticsearch geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00


Start with the necessary imports, a couple tweaks, and defining a "esql" query that will make easier to see the results of a query, by generating a Pandas or Geopandas dataframe.

In [2]:
import os
import io

import warnings

from elasticsearch import Elasticsearch
from elasticsearch import ElasticsearchWarning
from elasticsearch.exceptions import BadRequestError

import pandas as pd
import geopandas as gpd
from shapely import wkb

# Hide the warning when no LIMIT is passed on a ES|QL query
warnings.filterwarnings('ignore', category=ElasticsearchWarning)

# Allow wide columns
pd.set_option('display.max_colwidth', None)

# Convert Well-known Binary to Text
def wkb_to_wkt(wkb_bytes):
    if wkb_bytes is None:
        return None
    try:
        return wkb.loads(wkb_bytes).wkt
    except Exception as e:
        print(f"Error converting WKB: {wkb_bytes} - {e}")
        return None

# Generate a Pandas Dataframe or a Geopandas Dataframe from a ES|QL query
def esql(query, geometry_col:str = "geometry", use_arrow:bool = False):
    try:
        # Query ES and create a Pandas Dataframe
        if use_arrow:
            es_response = client.esql.query(query=query.strip(), format="arrow", columnar=True)
            df = es_response.to_pandas()
        else:
            es_response = client.esql.query(query=query.strip(), format="csv")
            df = pd.read_csv(io.StringIO(str(es_response)))

        # Promote to a Geopandas Dataframe if a "geometry" column
        if geometry_col in df.columns:
            if use_arrow:
                # Arrow geometries are transferred as WKB
                df[geometry_col] = df[geometry_col].apply(wkb_to_wkt)
            gs = gpd.GeoSeries.from_wkt(df[geometry_col])
            gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
            if geometry_col != "geometry":
                gdf.drop(columns="geometry")
            return gdf
        else:
            return df
    except BadRequestError as e:
        print("Something went wrong!")
        print(e.message)
        print("\r\n".join([c['reason'] for c in e.info['error']['root_cause']]))

Connect to Elasticsearch and print some cluster details

In [3]:
# Login details
ES_URL=os.getenv("ES_URL","https://foss4ge-lab.es.us-central1.gcp.cloud.es.io")
KB_URL=os.getenv("KB_URL","https://foss4ge-lab.kb.us-central1.gcp.cloud.es.io")

# API key that allows reading indices
ES_APIKEY=os.getenv("ES_APIKEY", "YlhrdDlwY0JPaUxuOUVMNlpHWDI6TG1LMXNFQTZQOXlGZUg5bFppaHN0UQ==")

# Load the client
client = Elasticsearch(hosts=[ES_URL], api_key=ES_APIKEY)

# Check the client
if client.ping():
  print("Connected to Elasticsearch")
  c_info = client.info()
  is_serverless = c_info['version']['build_flavor'] == 'serverless'

  # Print some cluster details
  print(f"Elasticsearch URL: {ES_URL}")
  print(f"Cluster name: {c_info['name']}")
  print(f"Version: {c_info['version']['number'] if not is_serverless else 'serverless'}")
  print("Number of documents indexed: ", client.count(index="*")['count'])
else:
  print("Connection failed")


Connected to Elasticsearch
Elasticsearch URL: https://foss4ge-lab.es.us-central1.gcp.cloud.es.io
Cluster name: instance-0000000001
Version: 9.0.3
Number of documents indexed:  2583076


## Syntax and API

### Basic syntax

<https://www.elastic.co/docs/reference/query-languages/esql/esql-syntax>

A ES|QL query is made of a `source` command that sets the data to retrieve and a list of processing commands starting with the pipe `|` character.

```text
source-command
| processing-command1
| processing-command2
```

A query can contain one line and multi line comments.

```
source-command           // Single line comment
| processing-command1    // another comment
/*
a multi
line comment in between
processing commands
*/
| processing-command2
```

About literals:

* Literals are duble quoted
* If a double quote is required in a literal, triple quotes can be used

```text
ROW name = """Indiana "Indy" Jones"""
```


### Query API

* [Documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql-rest)
* [Reference](https://www.elastic.co/docs/api/doc/elasticsearch/group/endpoint-esql)


Elasticsearch exposes the `_query` endpoint to execute ES|QL queries, with a `format` parameter to select between different output types as `csv`, `tsv`, `arrow`, `json`, etc.

As a `curl` command a request would be as:

```
curl -X POST \
  -H "Authorization: ApiKey $ES_APIKEY" \
  -H "Content-Type: application/json" \
  -d '{"query":"FROM places-* | STATS count = count(name) | LIMIT 1"}'\
  "$ES_URL/_query?pretty&format=txt"

     count     
---------------
230573
```

In the [Kibana Console](https://www.elastic.co/docs/explore-analyze/query-filter/tools/console):

```
POST /_query?format=txt
{
  "query": "FROM places-* | STATS count = count(name) | LIMIT 1"
}
```

And sending multiline queries is possible with triple quotes:

```text
POST /_query?format=txt
{
  "query": """
  FROM places-*
  | STATS count = count(name)
  | LIMIT 1
  """
}
```

A ES|QL API query can also include a filter using Elasticsearch DSL language:


```text
POST /_query?format=txt
{
  "query": """
  FROM places-*
  | STATS count = count(name)
  | LIMIT 1
  """,
  "filter": {
    "range": {
      "confidence": {
        "gte": 0.1,
        "lte": 1
      }
    }
  }
}
```

Other API endpoints available:

* [`_query/async`](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-esql-async-query): start, stop, and get results asynchronously
* [`_query/queries`](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-esql-list-queries): get details about running queries

## ES|QL sources: `ROW`, `SHOW`, `FROM`

In [4]:
# Creating a row directly, useful to test functions
esql('ROW a = 1, b = "two", c = null')

,a,b,c
0,1,two,NaN


In [5]:
# SHOW source returns Elasticsearch version
esql("SHOW INFO")

,version,date,hash
0,9.0.3,2025-06-18T22:09:56.772581489Z,cc7302afc8499e83262ba2ceaa96451681f0609d


In [6]:
# Basic query against all places indices
esql("FROM places-*")

,addresses.country,addresses.freeform,addresses.locality,addresses.postcode,addresses.region,alt_categories,brand,category,confidence,emails,geometry,id,name,phones,socials,source,updated,version,websites
0,KR,"언주로 325 (역삼동, 선인빌딩) 7층",Gangnam District,06226,NaN,library,NaN,test_preparation,0.337662,NaN,POINT (127.0275 37.49667),657a7f16-29e9-4b2b-a6ab-33f5ba96fa75,그린램프라이브러리 도곡1캠퍼스,NaN,https://www.facebook.com/100548728489874,meta,2025-06-02T07:00:00.000Z,1,NaN
1,KR,삼성로 82길 27,Gangnam District,06184,NaN,NaN,NaN,information_technology_company,0.672566,NaN,POINT (127.0275 37.49667),ddae7725-4d2b-4004-8e1b-19c7ac4c6b74,EBIT Korea,NaN,https://www.facebook.com/329373050268613,meta,2025-06-02T07:00:00.000Z,1,https://www.ebitglobal.ai/
2,KR,NaN,Gangnam District,NaN,NaN,NaN,NaN,travel_services,0.672566,NaN,POINT (127.0275 37.49667),430f987f-cb67-419d-9a4a-971d6fc62341,캠핑클럽,+827048581401,https://www.facebook.com/111617050973914,meta,2025-06-02T07:00:00.000Z,1,https://www.instagram.com/camping_club_/
3,KR,NaN,Gangnam District,NaN,NaN,NaN,NaN,bank_credit_union,0.331395,NaN,POINT (127.0275 37.49667),49f0918c-72d1-4778-81d5-f718e176dfbc,새희망드림팀,NaN,https://www.facebook.com/180521328469147,meta,2025-06-02T07:00:00.000Z,1,NaN
4,KR,도곡로135 수유빌딩2층 온라인피해365센터,Gangnam District,100744,NaN,NaN,NaN,lawyer,0.331395,NaN,POINT (127.0275 37.49667),fe28bbdb-6e7c-4745-ba38-adea4d3f3a04,사이버피해365센터,NaN,https://www.facebook.com/158633353995419,meta,2025-06-02T07:00:00.000Z,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NZ,10 Ra Ora Dr,Auckland,2013,AUK,"[boat_dealer, boat_rental_and_training]",NaN,boat_service_and_repair,0.978451,NaN,POINT (174.88203 -36.93979),61f7fa07-ac65-462b-aad6-0d78260eb735,Lloyd Stevenson Boatbuilders Ltd,+6492747680,https://www.facebook.com/458232080858568,meta,2025-06-02T07:00:00.000Z,1,http://www.lloydstevensonyachts.co.nz/
996,NZ,24 Ra Ora Dr,Auckland,2013,AUK,NaN,NaN,sign_making,0.814815,NaN,POINT (174.8819 -36.9396),c6b22dba-93ba-4d07-9eea-ca0c41f40554,Kiwi Street Signs,+6492734397,https://www.facebook.com/128486797818653,meta,2025-06-02T07:00:00.000Z,1,http://www.kiwistreets.co.nz/
997,NZ,11b Ra Ora Dr,Auckland,2013,AUK,professional_services,NaN,shopping,0.337662,NaN,POINT (174.88176 -36.93928),4815ba44-be22-4f41-83fd-4f0d589b7ea0,Pacific Aerials Ltd,+6492539450,https://www.facebook.com/615762301803902,meta,2025-06-02T07:00:00.000Z,1,http://www.pacificaerials.com/
998,NZ,1 Analie Pl,Auckland,2013,AUK,accommodation,NaN,linen,0.930061,NaN,POINT (174.88218 -36.93928),b67519c1-1931-4507-a69b-14219e280c8f,Home Vision Blinds,+6492741930,https://www.facebook.com/408171002557325,meta,2025-06-02T07:00:00.000Z,1,http://www.homevisionblinds.co.nz/


## Control the output: `LIMIT`

By default a ES|QL query result is limited to `1000` rows. Use `LIMIT` to reduce that number

In [7]:
# Basic query against all places indices,
# returing the first 5 rows (in no particular order)

esql("""
FROM places-*
| LIMIT 5
""")

,addresses.country,addresses.freeform,addresses.locality,addresses.postcode,addresses.region,alt_categories,brand,category,confidence,emails,geometry,id,name,phones,socials,source,updated,version,websites
0,KR,목동 924번지 KT빌딩 양천구 서울특별시 KR,NaN,07994,NaN,"[internet_cafe, restaurant]",NaN,coffee_shop,0.920482,NaN,POINT (126.87111 37.5294),2c622c3b-dead-4a19-a449-44d75a5c74c4,커피니,10226476964,https://www.facebook.com/308058162693325,meta,2025-06-02T07:00:00.000Z,1,http://coffeenie.co.kr
1,KR,목동 924,NaN,158-050,NaN,social_service_organizations,NaN,day_care_preschool,0.909824,NaN,POINT (126.87112 37.52945),6a7e3a2a-7c51-461f-adb5-6aadd358c998,KTIT 어린이집,82232191840,https://www.facebook.com/638952202794048,meta,2025-06-02T07:00:00.000Z,1,NaN
2,KR,목동 924,NaN,158-050,NaN,"[day_care_preschool, real_estate]",NaN,engineering_services,0.883117,NaN,POINT (126.87111 37.52946),6eda464f-3554-46f2-8d54-9a6c98033c8a,Ktit 본부 서비스기획팀,82232193127,https://www.facebook.com/253095285160961,meta,2025-06-02T07:00:00.000Z,1,NaN
3,KR,"Mokdongseo-ro, 201 kt정보전산센터 1층",NaN,07992,NaN,"[music_venue, theatre]",NaN,topic_concert_venue,0.492940,NaN,POINT (126.8709 37.52954),d2da76a2-ad59-426b-84b7-98d28f4a7e79,KT 챔버홀,8227398248,https://www.facebook.com/446274498749635,meta,2025-06-02T07:00:00.000Z,1,http://www.ktchamberhall.com/index_real.asp
4,KR,목동 923-12,NaN,07992,NaN,diner,NaN,caterer,0.909824,NaN,POINT (126.87092 37.52961),0ed8d561-3646-4703-a09c-07f91f2cd917,KT 목동지점,82226490060,https://www.facebook.com/122886037856031,meta,2025-06-02T07:00:00.000Z,1,NaN


## Change the output with `KEEP`, `RENAME`, and `SORT`

In [8]:
# Rename a field and only return a limited set of fields
esql("""
FROM places-*
| RENAME name as title
| KEEP title, category
| LIMIT 5
""")

,title,category
0,Avenidashow1,dance_club
1,Ccms - Morada Dos Sonhos,non_governmental_association
2,Sankio ltda,it_service_and_computer_repair
3,Paty arte,clothing_store
4,Igor Motos Customizado,professional_services


In [9]:
# KEEP also establishes the order of the columns returned,
# sometimes relevant for post-processing in client code
esql("""
FROM places-*
| RENAME name as title
| KEEP category, title
| LIMIT 5
""")

,category,title
0,dance_club,Avenidashow1
1,non_governmental_association,Ccms - Morada Dos Sonhos
2,it_service_and_computer_repair,Sankio ltda
3,clothing_store,Paty arte
4,professional_services,Igor Motos Customizado


In [10]:
# Once renamed, the previous identifier is not available anymore
esql("""
FROM places-*
| RENAME name as title
| KEEP name, category
| LIMIT 5
""")

Something went wrong!
verification_exception
Found 1 problem
line 3:8: Unknown column [name]


In [11]:
# Sort by a field, and reverse another
esql("""
FROM places-bosnia
| RENAME name AS title
| SORT category ASC, title DESC
| KEEP category, title
| LIMIT 5
""")

,category,title
0,abuse_and_addiction_treatment,Physio Ben
1,abuse_and_addiction_treatment,Odvikavanje
2,abuse_and_addiction_treatment,MedTim International
3,abuse_and_addiction_treatment,Laser centar
4,abuse_and_addiction_treatment,Klinika MedTiM


## Include metadata with `METADATA`

Use `METADATA` to get access to the `_index` and `_id`:

In [12]:
# Get also the source index using the METADATA keyword
esql("""
FROM places-* METADATA _index, _id
| KEEP _index, _id, name, category
| LIMIT 5
""")

,_index,_id,name,category
0,places-belem,65880021-e380-4737-8117-2b8cef9d444e,Avenidashow1,dance_club
1,places-belem,4007b4ea-f8d1-4f44-a4f6-72e7434f2e2b,Ccms - Morada Dos Sonhos,non_governmental_association
2,places-belem,5ee8b017-85d3-4d41-9b44-845bfe50be8e,Sankio ltda,it_service_and_computer_repair
3,places-belem,5306c52a-a35e-44b1-97e5-2fb3cc3c5cd2,Paty arte,clothing_store
4,places-belem,c5b84399-256a-4067-95fb-1ecf3a2075e5,Igor Motos Customizado,professional_services


## Filtering

In [13]:
# A basic filter
esql("""
FROM places-* METADATA _index
| RENAME _index as dataset
| WHERE name LIKE "*Burger*" //Exact text search
    AND category IN ("restaurant", "burger_restaurant") //Exact keyword search
    AND confidence < 0.3
| SORT confidence DESC
| KEEP dataset, name, category, confidence
| LIMIT 5
""")

,dataset,name,category,confidence
0,places-belem,Prime Burger food truck,burger_restaurant,0.296943
1,places-belem,Nick Burger,burger_restaurant,0.296943
2,places-bosnia,Burgers by Manzoni,burger_restaurant,0.296943
3,places-seoul,Burger Plant,burger_restaurant,0.296943
4,places-belem,Purple Burgers,burger_restaurant,0.296943


Truly leveraging Elasticsearch full text search features with the `MATCH` function.

[ES|QL documentation](https://www.elastic.co/docs/reference/query-languages/esql/functions-operators/search-functions#esql-match) | [DSL documentation](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax)

In [62]:
# Full text search with MATCH function
esql("""
FROM places-*
| WHERE MATCH(name, "snack BAR charlie", {"operator": "AND"})
| KEEP name
| LIMIT 1000
""")

,name
0,Charlie Bar and Snack


Sending queries in [Lucene](https://lucene.apache.org/) syntax with the `QSTR` function.

[ES|QL documentation](https://www.elastic.co/docs/reference/query-languages/esql/functions-operators/search-functions#esql-qstr) | [DSL documentation](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query) | [Query String syntax](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax)

In [60]:
# Full text search with Lucene syntax
# https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax
esql("""
FROM places-* METADATA _index
| WHERE QSTR("name: CHARLIE snack bar",
            {"fuzziness": 2,
            "minimum_should_match": 2,
            "phrase_slop": 3})
| KEEP _index, name
| LIMIT 1000
""")

,_index,name
0,places-bosnia,Caffee bar Charlie
1,places-bosnia,Charlie Bar
2,places-bosnia,Charlie Bar and Snack
3,places-bosnia,Caffe Bar Charlie Chaplin
4,places-bosnia,Charlie Tango Lounge Bar


---
>For more examples on how to do different text search operations with ES|QL check this [tutorial](https://www.elastic.co/guide/en/elasticsearch/reference/current/esql-search-tutorial.html)
---

## Aggregations with `STATS`

In [63]:
# STATS allows running aggrecations.
# In this count agg, no other data is available afterwards
esql("""
FROM ne_countries
| STATS counts = count(id)
""")

,counts
0,257


In [65]:
# When grouping by other fields, those are also available
# for further operations like sorting or filtering
esql("""
FROM ne_countries
| WHERE type in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent
| WHERE counts > 30
| SORT continent
| KEEP continent, counts //not necessary, only used to define order
| LIMIT 5
""")

,continent,counts
0,Africa,53
1,Asia,48
2,Europe,48


In [ ]:
# Aggregate: count by more than one grouping field
esql("""
FROM ne_countries
| WHERE type not in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent, type
| WHERE counts > 1
| SORT continent, type
| KEEP continent, type, counts
| LIMIT 50
""")

,continent,type,counts
0,Africa,Indeterminate,2
1,Asia,Dependency,3
2,Asia,Indeterminate,5
3,Europe,Disputed,2
4,North America,Dependency,12
5,North America,Indeterminate,2
6,Oceania,Dependency,12
7,Seven seas (open ocean),Dependency,5
8,South America,Indeterminate,2


## Computing new fields with `EVAL`

In [71]:
# Use EVAL to compute new fields
esql("""
FROM ne_countries
| WHERE gdp_md IS NOT NULL
    AND pop_est > 0
   // AND type IN ("Country", "Sovereign country")
| EVAL gdp_pop = ROUND(gdp_md * 1e6 / ( pop_est::double))::integer
| SORT gdp_pop DESC
| KEEP name, type, gdp_md, pop_est, gdp_pop
| LIMIT 10
""")

,name,type,gdp_md,pop_est,gdp_pop
0,Monaco,Sovereign country,7188,38964,184478
1,Liechtenstein,Sovereign country,6876,38019,180857
2,Bermuda,Dependency,7484,63918,117088
3,Luxembourg,Sovereign country,71104,619896,114703
4,Fr. S. Antarctic Lands,Dependency,16,140,114286
5,Isle of Man,Country,7491,84584,88563
6,Cayman Is.,Dependency,5517,64948,84945
7,Macao,Country,53859,640445,84096
8,Falkland Is.,Disputed,282,3398,82990
9,Switzerland,Sovereign country,703082,8574832,81994


## Joins

Traditionally, Elasticsearch has not been able to join datasets in a comfortable way for developers and analysts. This has changed with ES|QL and with the introduction of the `index.mode: lookup` setting.

More details on:

* [`LOOKUP JOIN` docs](https://www.elastic.co/docs/reference/query-languages/esql/esql-lookup-join)
* [`index.mode`](https://www.elastic.co/docs/reference/elasticsearch/index-settings/index-modules#index-mode-setting)

Our `ne_countries` index was created with that setting so we can now join that dataset with our places indices.

In [72]:
# Let's find outlier data in our places-* indices
esql(
"""
FROM places-*

// Aggregate all our places by country
| STATS counts = count(addresses.country) BY addresses.country

// Filter for only those groups with counts between 1 and 100
| WHERE counts > 1 AND counts < 50

// The lookup common field needs to be the same
| RENAME addresses.country AS iso_a2, counts AS places

// Run the LOOKUP JOIN by the iso_a2 field
| LOOKUP JOIN ne_countries ON iso_a2

// Merge records for repeated iso_a2 entries
| STATS places = SUM(places), names = VALUES(name) BY iso_a2

// Sort and print
| SORT places DESC
| KEEP iso_a2, places, names
"""
)

,iso_a2,places,names
0,AU,136,"[Australia, Indian Ocean Ter., Coral Sea Is., Ashmore and Cartier Is.]"
1,HU,29,Hungary
2,FR,28,"[France, Clipperton I.]"
3,SA,25,Saudi Arabia
4,VN,22,Vietnam
...,...,...,...
56,AZ,2,Azerbaijan
57,IE,2,Ireland
58,ZM,2,Zambia
59,BZ,2,Belize


## Wrap up

This is just the basics, there are plenty of functions and operators to process your datasets, please refer to the [documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql) for further details.